In [1]:
import pandas as pd
import numpy as np

import pyomo.environ as pyo

import gurobipy as gp
from gurobipy import GRB

import openpyxl

from icecream import ic


In [2]:
###
### RETRIEVE AND CLEAN DATA
###
file_path = "Railway services-2024.xlsx"

try:
    df = pd.read_excel(io=file_path, engine="openpyxl")
    ic(df.head())

except FileNotFoundError:
    print(f"File '{file_path}' not found. Please check the file path.")

# Data cleaning/ enhancing

# Removing unneccesary spaces from 'From' and 'To' columns. 
df['From'] = df['From'].str.strip()
df['To'] = df['To'].str.strip()

# Adding an indicator when the trip is located on line 400
indicator_line_400 = np.where(df['Line'] == 400.0, 1, 0)
df["line_400"] = indicator_line_400


ic| df.head():    Trip Departure Time Arrival Time From To  Demand(μ)  Demand(σ)  Line
               0     1       07:00:00     07:46:00    M  A        327         64   100
               1     2       07:00:00     07:57:00    H  M        936        308   800
               2     3       07:02:00     07:22:00    M  B        461        252   200
               3     4       07:04:00     08:03:00    M  J        428         10  1000
               4     5       07:06:00     08:13:00    M  F        449        124   600


In [3]:
### 
### CREATING THE MODEL
###

##
## Initial values:
##
train_type = [
    "OC", 
    "OH"
    ]
cost_train_type = {
    "OC": 260,
    "OH": 210
}
length_train_type = {
    "OC": 100, 
    "OH": 70
    }
capacity_train_type = {
    "OC": 620, 
    "OH": 420
    }

#
# Data manipulated initial values:
#
cross_section = set(
    df['From'].values + df['To'].values
    ) #NOTE: Not used

trips_using_line400 = ((df.loc[df['line_400'] == 1]).index + 1).to_list() # +1 because the indexes start with 0, but the trips at 1. 


###
### Creation of the Model:
###
def create_general_model():
    m = pyo.ConcreteModel()

    ##
    ## CREATION OF SETS
    ##
    m.trips = pyo.Set(
        initialize = df['Trip']
        )

    m.train_type = pyo.Set(
        initialize = train_type
        )

    ##
    ## CREATION OF PARAMETERS
    ##
    m.cost_train_type = pyo.Param(
        m.train_type, 
        initialize = cost_train_type
    )

    m.length_train_type = pyo.Param(
        m.train_type, 
        initialize = length_train_type
        )

    m.capacity_train_type = pyo.Param(
        m.train_type, 
        initialize = capacity_train_type
        )
    
    #
    # Parameters created using functions:
    #
    m.trips_using_line400 = pyo.Param(
        m.trips,  # Assuming model.cross_section contains all possible cross sections
        initialize={trip: 1 if trip in trips_using_line400 else 0 for trip in m.trips}
    )

    m.passengers_per_trip = pyo.Param(
        m.trips, 
        initialize = {trip: df['Demand(μ)'].loc[trip - 1] for trip in m.trips} # -1 because the trips start with trip 1, but the df with index 0. So, loc[200] results in an error, because there are only 199 records. 
    )

    return m


m = create_general_model()


In [4]:
###
### ADDING SPECIFICS FOR MODEL1
###

##
## Variable specific for Model1
##
def specific_variables_Model1(m):
    #
    # Index set for allocation variable model1
    #
    m.index_set_allocation = pyo.Set(
        initialize = m.trips * m.train_type
        )

    #
    # Variable for allocation in Model1
    #
    # the variable represents the number of trains per type allocated to a trip.
    #
    m.allocation_train_numbers = pyo.Var(
        m.index_set_allocation, 
        domain = pyo.NonNegativeIntegers, 
        name = 'train_allocation', 
        doc = 'The number of trains of a certain type allocated to a cross-section'
    )
    return m

##
## Constraints specific for Model1
##
def specific_constraints_Model1(m):
    #
    # Rule for restricting the combined train length. 
    #
    def rule_maximum_length(
            m, 
            trip
            ):
        combined_train_length = sum(
            m.length_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return combined_train_length <= 300 - 100 * m.trips_using_line400[(trip)]
    m.constr_maximum_length = pyo.Constraint(
        m.trips, 
        rule = rule_maximum_length
        )
    
    #
    # Rule for enforcing the ability to accomodate all passengers. 
    #
    def rule_passenger_limit(
            m, 
            trip
            ):
        available_capacity = sum(
            m.capacity_train_type[(train_type)] * m.allocation_train_numbers[(trip, train_type)]
            for train_type in m.train_type
        )
        return available_capacity >= int(m.passengers_per_trip[(trip)])
    m.constr_passenger_limit = pyo.Constraint(
        m.trips, 
        rule = rule_passenger_limit
        )
    
    # FIXME: Should this be with [0.75, 1] or with [1, 1.25] Same for the other Model
    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 1 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            ) <= 1.25 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 1 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OH"
            ) <= 1.25 * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips 
                for train_type in m.train_type if train_type == "OC"
            )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    return m


##
## Objective rule specific for Model1
##
def specific_objective_Model1(m):
    #
    # Objective rule to minimize the total cost of all trains. 
    #
    def obj_minimize_total_cost(m):
        total_cost = sum(
            m.cost_train_type[(train_type)] * sum(
                m.allocation_train_numbers[(trip, train_type)] 
                for trip in m.trips
            )
            for train_type in m.train_type
        )
        return total_cost
    m.objective = pyo.Objective(
        rule=obj_minimize_total_cost,
        sense=pyo.minimize
    )
    return m


##
## Add all variables, constraints and objectives to Model1
##
new_model = create_general_model()
model1 = specific_variables_Model1(m=new_model)
model1 = specific_constraints_Model1(m=model1)
model1 = specific_objective_Model1(m=model1)


In [5]:
###
### ADDING SPECIFICS FOR MODEL2
###


#
# Obtain all compositions
#
def get_compositions():
    # Define the lengths of each train type
    OH_length = 100
    OC_length = 70

    OH_cap = 420
    OC_cap = 620

    # Define the maximum length of a train
    max_train_length = 300

    # Generate all possible combinations of OH and OC trains
    all_combinations = {}

    for num_OH in range(max_train_length // OH_length + 1):
        for num_OC in range(max_train_length // OC_length + 1):

            length = num_OH * OH_length + num_OC * OC_length
            if length <= max_train_length:
                combination = []
                if num_OH > 0:  # Append 'OH' only if num_OH is greater than 0
                    combination.extend(['OH'] * num_OH)
                if num_OC > 0:  # Append 'OC' only if num_OC is greater than 0
                    combination.extend(['OC'] * num_OC)

                cap = num_OH * OH_cap + num_OC * OC_cap

                if length != 0:
                    all_combinations[str(combination)] = [length, cap, num_OC, num_OH]

    return all_combinations

##
## Parameter specific for Model2
##
def specific_parameter_Model2(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())

    #
    # Set containing the index for the quantification of the number of train types per composition.
    #
    m.index_quantify_compositions = pyo.Set(
        initialize= m.compositions * m.train_type
    )

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    m.quantify_compositions = pyo.Param(
        m.index_quantify_compositions,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in m.compositions for train_type in m.train_type
        }
    )

    return m


##
## Variable specific for Model2
##
def specific_variables_Model2(m):
    # 
    # Index set for allocation variable model1
    # 
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions)


    # 
    # Variable for allocation in Model1
    # 
    # the binary variable indicates the composition used on a trip.
    # 
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip",
        doc = "Indicates which composition is used on a trip"
    )

    return m


## 
## Constraints specific for Model2
## 
def specific_constraints_Model2(m):
    # NOTE: CORRECT
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip):
        return sum(
            m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1
    )
    def rule_difference_between_number_of_train_types2(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for trip in m.trips
        )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2
    )

    #
    # Rule for the length of the train. 
    #
    def rule_maximum_length(m, trip):
        return sum(
            m.length_train_type[(train_type)] * m.quantify_compositions[(composition, train_type)] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for train_type in m.train_type
        ) <= 300 - 100 * m.trips_using_line400[(trip)]  
    m.constr_maximum_length = pyo.Constraint(m.trips, rule=rule_maximum_length)

    #
    # Rule for the capacity of a train
    #
    def rule_capacity_limit(m, trip):
        return sum(
            m.capacity_train_type[(train_type)] * m.quantify_compositions[(composition, train_type)] * m.allocation_compositions[(trip, composition)]
            for composition in m.compositions
            for train_type in m.train_type
        ) >= m.passengers_per_trip[(trip)]
    m.constr_capacity_limit = pyo.Constraint(m.trips, rule=rule_capacity_limit)

    return m


    
def specific_objective_Model2(m):
    m.cost_train_type.pprint()
    def obj_minimize_total_cost(m):
        return m.cost_train_type["OH"] * sum(
                m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            ) + (m.cost_train_type["OC"]) * sum(
                m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition)]
                for composition in m.compositions
                for trip in m.trips
            )
    m.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)
    m.objective.pprint()
    return m

new_model = create_general_model()
model2 = specific_parameter_Model2(m=new_model)
model2 = specific_variables_Model2(m=model2)
model2 = specific_constraints_Model2(m=model2)
model2 = specific_objective_Model2(m=model2)

cost_train_type : Size=2, Index=train_type, Domain=Any, Default=None, Mutable=False
    Key : Value
     OC :   260
     OH :   210
objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 210*(0*allocation_compositions[1,"['OC']"] + 0*allocation_compositions[2,"['OC']"] + 0*allocation_compositions[3,"['OC']"] + 0*allocation_compositions[4,"['OC']"] + 0*allocation_compositions[5,"['OC']"] + 0*allocation_compositions[6,"['OC']"] + 0*allocation_compositions[7,"['OC']"] + 0*allocation_compositions[8,"['OC']"] + 0*allocation_compositions[9,"['OC']"] + 0*allocation_compositions[10,"['OC']"] + 0*allocation_compositions[11,"['OC']"] + 0*allocation_compositions[12,"['OC']"] + 0*allocation_compositions[13,"['OC']"] + 0*allocation_compositions[14,"['OC']"] + 0*allocation_compositions[15,"['OC']"] + 0*allocation_compositions[16,"['OC']"] + 0*allocation_compositions[17,"['OC']"] + 0*allocation_compositions[18,"['OC']"] + 0*allocation_comp

In [10]:
###
### ADDING ADDITIONAL REQUIREMENTS FOR MODEL3
###

#
# Obtain all compositions
#
def get_compositions():
    # Define the lengths of each train type
    OH_length = 100
    OC_length = 70

    OH_cap = 420
    OC_cap = 620

    # Define the maximum length of a train
    max_train_length = 300

    # Generate all possible combinations of OH and OC trains
    all_combinations = {}

    for num_OH in range(max_train_length // OH_length + 1):
        for num_OC in range(max_train_length // OC_length + 1):

            length = num_OH * OH_length + num_OC * OC_length
            if length <= max_train_length:
                combination = []
                if num_OH > 0:  # Append 'OH' only if num_OH is greater than 0
                    combination.extend(['OH'] * num_OH)
                if num_OC > 0:  # Append 'OC' only if num_OC is greater than 0
                    combination.extend(['OC'] * num_OC)

                cap = num_OH * OH_cap + num_OC * OC_cap

                if length != 0:
                    all_combinations[str(combination)] = [length, cap, num_OC, num_OH]

    return all_combinations

##
## Parameter specific for Model2
##
def specific_parameter_Model3(m):
    #
    # Set containing all compositions
    #
    compositions = get_compositions()
    #ic(compositions)

    m.compositions = pyo.Set(initialize = compositions.keys())
    
    m.days = pyo.Set(initialize = list(range(1, 251)))

    #
    # Set containing the index for the quantification of the number of train types per composition.
    #
    m.index_quantify_compositions = pyo.Set(
        initialize= m.compositions * m.train_type * m.days
    )

    #
    # Parameter indicating number of 'OC' and 'OH' within a composition
    #
    m.quantify_compositions = pyo.Param(
        m.compositions * m.train_type,
        initialize = {
            (composition, train_type): compositions[composition][2] if train_type == "OC" else compositions[composition][3]
            for composition in m.compositions for train_type in m.train_type
        }
    )

    m.passengers_per_trip = pyo.Param(
        m.trips * m.days, 
        initialize = {
            (trip, day): np.random.normal(
                df['Demand(μ)'].loc[trip - 1], 
                df['Demand(σ)'].loc[trip - 1]
                ) 
            for trip in m.trips 
            for day in m.days
            } # -1 because the trips start with trip 1, but the df with index 0. So, loc[200] results in an error, because there are only 199 records. 
    )
    #m.passengers_per_trip.pprint()

    m.demand_upperbound = 5000

    return m


##
## Variable specific for Model3
##
def specific_variables_Model3(m):
    # 
    # Index set for allocation variable model1
    # 
    m.index_set_allocation = pyo.Set(initialize = m.trips * m.compositions * m.days)


    # 
    # Variable for allocation in Model1
    # 
    # the binary variable indicates the composition used on a trip.
    # 
    m.allocation_compositions = pyo.Var(
        m.index_set_allocation,
        domain = pyo.Binary,
        name = "Specific composition on trip on a day",
        doc = "Indicates which composition is used on a trip on a specific day"
    )

    m.binairy_capacity_satisfied = pyo.Var(
        m.index_set_allocation, 
        domain = pyo.Binary, 
        name = "Indicates whether the composition on a trip on a day is able to satisfy demand"
    )

    return m


## 
## Constraints specific for Model3
## 
def specific_constraints_Model3(m):
    # NOTE: CORRECT
    # Rule that ensures a trip can only have a single composition
    #
    def rule_one_composition_per_trip(m, trip, day):
        return sum(
            m.allocation_compositions[(trip, composition, day)]
            for composition in m.compositions
        ) == 1
    m.constr_one_composition_per_trip = pyo.Constraint(
        m.trips, 
        m.days,
        rule=rule_one_composition_per_trip
    )

    #
    # Rules for limiting the 'favorate' train type to be max 0.25% higher than the less preferred. 
    #
    def rule_difference_between_number_of_train_types1_3(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition, day)]
            for composition in m.compositions
            for trip in m.trips
            for day in m.days
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition, day)]
            for composition in m.compositions
            for trip in m.trips
            for day in m.days
        )
    m.constr_difference_between_number_of_train_types1 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types1_3
    )
    def rule_difference_between_number_of_train_types2_3(m):
        return 1 * sum(
            m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition, day)]
            for composition in m.compositions
            for trip in m.trips
            for day in m.days
        ) <= 1.25 * sum(
            m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition, day)]
            for composition in m.compositions
            for trip in m.trips
            for day in m.days
        )
    m.constr_difference_between_number_of_train_types2 = pyo.Constraint(
        rule=rule_difference_between_number_of_train_types2_3
    )

    #
    # Rule for the length of the train. 
    #
    def rule_maximum_length_3(m, trip, day):
        return sum(
            m.length_train_type[(train_type)] * m.quantify_compositions[(composition, train_type)] * m.allocation_compositions[(trip, composition, day)]
            for composition in m.compositions
            for train_type in m.train_type
        ) <= 300 - 100 * m.trips_using_line400[(trip)]  
    m.constr_maximum_length = pyo.Constraint(
        m.trips, 
        m.days, 
        rule=rule_maximum_length_3 
        )

    #
    # Rules for the definition of the indicator for satisfied capacity
    #
    def rule_capacity_satisfied1(m, trip, composition, day):
        return m.passengers_per_trip[(trip, day)] - sum(
            m.quantify_compositions[(composition, train_type1)] * m.allocation_compositions[(trip, composition, day)] 
            for train_type1 in m.train_type
            ) >= - m.demand_upperbound * m.binairy_capacity_satisfied[(trip, composition, day)]
    m.constr_capacity_satisfied1 = pyo.Constraint(
        m.trips, 
        m.compositions, 
        m.days,
        rule = rule_capacity_satisfied1
    )
    def rule_capacity_satisfied2(m, trip, composition, day):
        return m.passengers_per_trip[(trip, day)] - sum(
            m.quantify_compositions[(composition, train_type1)] * m.allocation_compositions[(trip, composition, day)] 
            for train_type1 in m.train_type
            ) <= m.demand_upperbound * (1 - m.binairy_capacity_satisfied[(trip, composition, day)])
    m.constr_capacity_satisfied2 = pyo.Constraint(
        m.trips, 
        m.compositions, 
        m.days,
        rule = rule_capacity_satisfied2
    )

    #
    # Rule such that each trip has enough capacity for at least 125 days.
    #
    def rule_days_satisfied(m, trip):
        return sum(
                m.binairy_capacity_satisfied[(trip, composition, day)] 
                for composition in m.compositions 
                for day in m.days
            ) >= 125
    m.constr_days_satisfied = pyo.Constraint(
        m.trips, 
        rule = rule_days_satisfied
    )

    #
    # Rule such that at least 81% of the time the trips have enough capacity. 
    #
    def rule_days_satisfied(m):
        return sum(
                m.binairy_capacity_satisfied[(trip, composition, day)] 
                for trip in m.trips
                for composition in m.compositions 
                for day in m.days
            ) >= 0.81 * 50000
    m.constr_days_satisfied = pyo.Constraint(
        rule = rule_days_satisfied
    )

    return m


#
# Objective function for minimizing total costs.
#
def specific_objective_Model3(m):
    m.cost_train_type.pprint()
    def obj_minimize_total_cost(m):
        return m.cost_train_type["OH"] * sum(
                m.quantify_compositions[(composition, "OH")] * m.allocation_compositions[(trip, composition, day)]
                for composition in m.compositions
                for trip in m.trips
                for day in m.days
            ) + (m.cost_train_type["OC"]) * sum(
                m.quantify_compositions[(composition, "OC")] * m.allocation_compositions[(trip, composition, day)]
                for composition in m.compositions
                for trip in m.trips
                for day in m.days
            )
    m.objective = pyo.Objective(rule=obj_minimize_total_cost, sense=pyo.minimize)
    #m.objective.pprint()
    
    return m

new_model = create_general_model()
model3 = specific_parameter_Model3(m=new_model)
model3 = specific_variables_Model3(m=model3)
model3 = specific_constraints_Model3(m=model3)
model3 = specific_objective_Model3(m=model3)

(type=<class 'pyomo.core.base.param.IndexedParam'>) on block unknown with a
new Component (type=<class 'pyomo.core.base.param.IndexedParam'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
cost_train_type : Size=2, Index=train_type, Domain=Any, Default=None, Mutable=False
    Key : Value
     OC :   260
     OH :   210


In [11]:
###
### CREATING THE SOLVING STRUCTURE
###

##
## Solve a specific model
##
def solve_model(model_to_solve, time_limit: int = 90, print_info: bool = True):
    ##
    ## Selecting the solver
    ##
    solver = pyo.SolverFactory('cbc') #FIXME: Either 'cbc' or 'glpk': glpk for gurobi (pip install cbcpy or pip install gurobipy)

    #
    # Solve the model
    #
    results = solver.solve(
        model_to_solve, 
        options={'seconds': time_limit}
        )#, tee=True)

    #
    # Check the solver status
    #
    if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
        print("Solver terminated successfully. Model is feasible.")
    elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
        print("Solver terminated: Model is infeasible.")
    else:
        print("Solver terminated with non-optimal solution.")

    #
    # Output the information from the solver if necessary. 
    #
    if print_info:
        ic(results)
        ic(results.solver.time)
    
    return results

##
## Obtain solution from Model1
##
def get_solution_df_Model1(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for train_type in model.train_type:
            solution_dict[(trip, train_type)] = model.allocation_train_numbers[(trip, train_type)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.train_type)

    for trip in model.trips:
        for train_type in model.train_type:
            solution_DF.loc[trip, train_type] = solution_dict.get((trip, train_type), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
      
        
    return solution_DF


##
## Obtain solution from Model2
##
def get_solution_df_Model2(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            solution_dict[(trip, composition)] = model.allocation_compositions[(trip, composition)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=model.trips, columns=model.compositions)

    for trip in model.trips:
        for composition in model.compositions:
            solution_DF.loc[trip, composition] = solution_dict.get((trip, composition), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
    
    return solution_DF



## FIXME: The variable should most likely not be dependent on days, since that would defeat the purpose of the capacity constraints. 
## 
## Obtain solution from Model3
##
def get_solution_df_Model3(model, display_solution_df: bool = False):
    #
    # Use a dictionary to store all information from the decision variable
    #
    solution_dict = {}

    for trip in model.trips:
        for composition in model.compositions:
            for day in model.days:
                solution_dict[(trip, composition, day)] = model.allocation_compositions[(trip, composition, day)].value

    #
    # Create and fill a pandas dataframe to store the data from the dictionairy in a visually pleasing way.
    #
    solution_DF = pd.DataFrame(index=(model.trips, model.compositions), columns=model.days)

    for trip in model.trips:
        for composition in model.compositions:
            for day in model.days:
                solution_DF.loc[(trip, composition), day] = solution_dict.get((trip, composition, day), np.nan)

    #
    # Display the solution dataframe
    #
    if display_solution_df:
        ic(solution_DF)
    
    return solution_DF



In [12]:
###
### Solving the actual models
###

#
# Solving Model1
#
solve_model(model_to_solve=model1, time_limit=1)
solution1 = get_solution_df_Model1(model=model1, display_solution_df=True)


#
# Solving Model2
#
solve_model(model_to_solve=model2, time_limit=1)
solution2 = get_solution_df_Model2(model=model2, display_solution_df=True)

# 
# Solving Model3
#
solve_model(model_to_solve=model3, time_limit=90)
solution3 = get_solution_df_Model3(model=model3, display_solution_df=True)

containing a solution


ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 72917.778, 'Upper bound': 72940.0, 'Number of objectives': 1, 'Number of constraints': 402, 'Number of variables': 400, 'Number of binary variables': 0, 'Number of integer variables': 400, 'Number of nonzeros': 400, 'Sense': 'minimize'}], 'Solver': [{'Status': 'aborted', 'User time': -1.0, 'System time': 1.0, 'Wallclock time': 1.54, 'Termination condition': 'maxTimeLimit', 'Termination message': 'Optimization terminated because the time expended exceeded the value specified in the seconds parameter.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 360, 'Number of created subproblems': 360}, 'Black box': {'Number of iterations': 1852}}, 'Error rc': 0, 'Time': 1.594486951828003}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 1.594486951828003
ic| solution_DF:       OC   OH
                 1    0.0  1.0
                 2    1.0  1.

Solver terminated with non-optimal solution.


ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 72940.0, 'Upper bound': 72940.0, 'Number of objectives': 1, 'Number of constraints': 602, 'Number of variables': 2000, 'Number of binary variables': 2000, 'Number of integer variables': 2000, 'Number of nonzeros': 2000, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.87, 'Wallclock time': 0.93, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 50, 'Number of created subproblems': 50}, 'Black box': {'Number of iterations': 186}}, 'Error rc': 0, 'Time': 0.9453697204589844}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 0.9453697204589844
ic| solution_DF:     ['OC'] ['OC', 'OC'] ['OC', 'OC', 'OC'] ['OC', 'OC', 'OC', 'OC'] ['OH']  \
             

Solver terminated successfully. Model is feasible.


 x 10 columns]


model.name="unknown";
    - termination condition: infeasible
    - message from solver: <undefined>


ic| results: {'Problem': [{'Name': 'unknown', 'Lower bound': 11611100.0, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 588487, 'Number of variables': 988330, 'Number of binary variables': 1000000, 'Number of integer variables': 1000000, 'Number of nonzeros': 500000, 'Sense': 'minimize'}], 'Solver': [{'Status': 'warning', 'User time': -1.0, 'System time': 246.29, 'Wallclock time': 250.24, 'Termination condition': 'infeasible', 'Termination message': 'Model was proven to be infeasible.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}}, 'Error rc': 0, 'Time': 250.4084289073944}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}
ic| results.solver.time: 250.4084289073944


Solver terminated: Model is infeasible.


TypeError: unhashable type: 'OrderedScalarSet'

In [ ]:
#
# CHECK FEASABILITY MODEL2
#






#
# CHECK FEASABILITY MODEL2
#

# Check allocation to specific train types. 
total_composition_sol2 = solution2.sum(axis=0)
total_composition_sol2 = total_composition_sol2[total_composition_sol2 != 0.0]
ic(total_composition_sol2)

total_alloc_per_trip = solution2.sum(axis=1)
total_alloc_per_trip = total_alloc_per_trip[total_alloc_per_trip != 1.0]
ic(total_alloc_per_trip)

ic((111/89 - 1) <= 0.25)


dataframe = pd.DataFrame(index=model3.trips, columns=model3.days)

for trip in model3.trips:
    for day in model3.days:
        dataframe.loc[trip, day] = model3.passengers_per_trip[(trip, day)]


dataframe = dataframe.max(axis=0)

ic(dataframe[(dataframe > 2000)])
ic(df[df['line_400'] == 1])
ic(get_compositions())



ic| total_composition_sol2: ['OC']                55.0
                            ['OC', 'OC']          14.0
                            ['OH']                39.0
                            ['OH', 'OC']          47.0
                            ['OH', 'OC', 'OC']     2.0
                            ['OH', 'OH']          37.0
                            ['OH', 'OH', 'OC']     6.0
                            dtype: object
ic| total_alloc_per_trip: Series([], dtype: object)
ic| (111/89 - 1) <= 0.25: True
ic| dataframe[(dataframe>2000)]: 9      2007.768057
                                 22     2326.837127
                                 24     2053.804299
                                 26     2162.650896
                                 32     2173.611295
                                 45      2493.74045
                                 47     2292.821637
                                 51     2021.700129
                                 68     2277.956044
                      

{"['OC']": [70, 620, 1, 0],
 "['OC', 'OC']": [140, 1240, 2, 0],
 "['OC', 'OC', 'OC']": [210, 1860, 3, 0],
 "['OC', 'OC', 'OC', 'OC']": [280, 2480, 4, 0],
 "['OH']": [100, 420, 0, 1],
 "['OH', 'OC']": [170, 1040, 1, 1],
 "['OH', 'OC', 'OC']": [240, 1660, 2, 1],
 "['OH', 'OH']": [200, 840, 0, 2],
 "['OH', 'OH', 'OC']": [270, 1460, 1, 2],
 "['OH', 'OH', 'OH']": [300, 1260, 0, 3]}